# England residential UPRNs : changes from 2020-2024 (Maps)
### Please run the cells in sequence!

Author - Samantha Iacob

Last updated - 19/4/2024

To-do: fix the panel output!!!!


The cells from here on will require geopackages to be installed on your machine! A guide to installing the modules you'll need is located [here on the ONS geospatial training repository on GitHub](https://github.com/ONSgeo/geospatial-training/blob/master/_docs/guides/python_install_anaconda.md).

In [9]:
#importing modules which I might use - these are installed in the training link above
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import mapclassify
from mapclassify import Quantiles, UserDefined
from matplotlib import cm
import numpy as np
from pathlib import Path
%matplotlib inline
BASE = Path('.').resolve() #Set path
import panel as pn
pn.extension()

In [ ]:
## import folium # additional modules
import geopandas as gpd
from pyproj import CRS
import requests
from folium.features import GeoJsonPopup, GeoJsonTooltip

In [11]:
#Importing a shapefile containing a simplified map of England's regions with data on UPRNs since 2020 included.
map_gdf = gpd.read_file('data/UPRN_e_regions_2023_base.shp') 
#map_gdf

In [12]:
#Prototype map
while True:
    year = (input('Enter a year between 2021-2024 inclusive: '))
    try:
        year = int(year)
    except: 
        pass
    else: 
        if year in [2021, 2022, 2023, 2024]:
            break

#Viridis chosen for colormap as designed to be viewed correctly by people with common forms of color blindness, 
#and because it looks nice on a light-colored map (I looked at almost all options!)
#other considerations were that some colors are perceived as negative - red or dark grey might imply that something
#is 'wrong' with areas with low UPRN growth whereas I do not know whether that is the case.

m = map_gdf[map_gdf.Year == year].explore("change_pct",
                                     cmap = "viridis",
                                     tiles = "CartoDB positron",
                                     tooltip = ["RGN23NM", "UPRN_count", "UPRN_2020", "change_rnd"],
                                     scheme = 'EqualInterval',
                                     k = 10,
                                     #style_kwds = {'stroke' : True, 'color' : "Black", 'weight' : 2, 'fill' : False},
                                     tooltip_kwds = {'aliases' : ["Region:", 
                                                                  f"{year} residential UPRNs:", 
                                                                  "2020 residential UPRNs:", 
                                                                  "% change since 2020:"]
                                                    },
                                      legend_kwds = {'caption' : 'Change (%) since 2020'},
                                          map_kwds = {'dragging' : False, 
                                                      'scrollWheelZoom' : False}, # locked because it's nice for the user!
                                          zoom_control=False # this gets annoying
                                                                          )
folium_pane = pn.pane.plot.Folium(m, height = 1000)
folium_pane

Enter a year between 2021-2024 inclusive: 2021


Folium(Map, height=1000)

In [13]:
#Creating a control for the dash
year_slider = pn.widgets.IntSlider(name='Year', value=2021, start=2021, end=2024)
#display(year_slider)

In [14]:
#Defining a function which creates the map and passes it back to panel as a pane.

def year_map(map_gdf):
    
    m = map_gdf.explore("change_pct",
                        cmap = "viridis",
                        tiles = "CartoDB positron",
                        tooltip = ["RGN23NM", "UPRN_count", "UPRN_2020", "change_rnd"],
                        scheme = 'EqualInterval',
                        k = 4,
                        #style_kwds = {'stroke' : True, 'color' : "Black", 'weight' : 2, 'fill' : False},
                        tooltip_kwds = {'aliases' : ["Region:", 
                                                    f"{year} residential UPRNs:", 
                                                    "2020 residential UPRNs:", 
                                                    "% change since 2020:"]
                                        },
                        legend_kwds = {'caption' : 'Change (%) since 2020'},
                        map_kwds = {'dragging' : False, 
                                    'scrollWheelZoom' : False}, # locked because it's nice for the user!
                        zoom_control=False # this gets annoying
                        )
    return pn.Column(
        pn.pane.HTML(m._repr_html_(), sizing_mode = "scale_both"), # :)
    )


In [15]:
#Function describing what changing the year slider does (creates a new DF just for that year)

def year_change(df = map_gdf, year = year_slider):
    new_df = df[df.Year == year]
    new_df.reset_index(drop = True, inplace = True)
    
    if len(new_df) == 0:
        return '### No data found'
    else:
        return year_map(new_df)

In [16]:
#generating the dash

layout = pn.interact(year_change)

dash = pn.Column(pn.Row('### Percentage change in residential UPRNs (baseline year: 2020)', layout[0]), layout[1])

dash

Column
    [0] Row
        [0] Markdown(str)
        [1] Column
            [0] IntSlider(end=2024, name='Year', start=2021, value=2021)
    [1] Row
        [0] Column
            [0] HTML(str, sizing_mode='scale_both')

In [17]:
#launches the dash in a new window!

dash.show()


Launching server at http://localhost:55374
